# 4. Named Entity Recognition

In [1]:
%run __init__.py

In [2]:
import pandas as pd

GIT_FILE_PATH = os.path.join(NOTEBOOK_1_RESULTS_DIR, 'git_dataframe.pkl')

git_df = pd.read_pickle(GIT_FILE_PATH)

In [3]:
git_repositories = git_df['full_text_cleaned'].values

In [4]:
MAX_CHAR_LENGTH = 100000

git_repositories = [r[:MAX_CHAR_LENGTH] for r in git_repositories]

In [5]:
git_repositories[21]

'Translate neo4j query output to YAML file suitable for importing to NCATS reasoner. neo4j-to-reasoner. Translate neo4j query output to YAML file suitable for importing to NCATS reasoner Authentication NOTES README cypher to reasoner driver q1 driver q2 import command lowercase nodes orphadata2cui chlorcyclizine asthma 10000 imatinib asthma full2 neo4j output snippet print edges q1 neo4j to reasoner q2 neo4j to reasoner reset database q1 disease list cui q2 drugandcondition list cui search umls sum pmids data index q1 results q2 results'

In [7]:
import en_core_web_md

from herc_common.text import NamedEntityRecognizer

ner = NamedEntityRecognizer(en_core_web_md)

In [8]:
import en_core_sci_lg

ner_2 = NamedEntityRecognizer(en_core_sci_lg)

In [9]:
text = git_repositories[21]

ents = ner.transform([text])
ents

[['NCATS', 'NCATS', 'Authentication', 'orphadata2cui', '10000', 'umls']]

In [10]:
ner.visualize_entities(text)

In [11]:
ner_2.transform([text])

[['Translate',
  'YAML',
  'importing',
  'NCATS reasoner',
  'Translate neo4j query',
  'YAML',
  'importing',
  'NCATS reasoner Authentication NOTES README cypher',
  'reasoner',
  'lowercase nodes',
  'chlorcyclizine asthma',
  'imatinib',
  'asthma',
  'full2']]

In [12]:
ner_2.visualize_entities(text)

## Saving results
Finally, we are going to save the named entity recognizer class with the parameters that we have selected for further use in the following phases.

In [13]:
from herc_common.utils import save_object

output_filename = "ner_system.pkl"
save_object(ner_2, os.path.join(NOTEBOOK_4_RESULTS_DIR, output_filename))